# Inner join - intuition

Dans la partie précédente, on a découvert les cross joins.

On a démarré par les cross joins parce que ça permet de bâtir de l'intuition sur les autres formes de joins.

Dans ce notebook, on va découvrir que tous les autres joins peuvent être imaginés comme des cross_joins avec un filtre

In [1]:
import pandas as pd
import duckdb
import io

## Cross join avec plusieurs colonnes dans la table

Imaginons qu'on ait deux tables, salaries & seniorities:

In [2]:
csv = '''
salary,employee_id
2000,1
2500,2
2200,3
'''

csv2 = '''
employee_id,seniority
1,2ans
2,4ans
'''


salaries = pd.read_csv(io.StringIO(csv))
seniorities = pd.read_csv(io.StringIO(csv2))

In [3]:
display(salaries)
seniorities

,salary,employee_id
0,2000,1
1,2500,2
2,2200,3


,employee_id,seniority
0,1,2ans
1,2,4ans


Comme ces deux tables contiennent respectivement 3 entrées et 2 entrées, on s'attends à ce qu'un cross join nous donne une table avec 6 entrées:

<img src="images/cross_join_before_filter.png" />

In [4]:
cross_join_query = """
SELECT * FROM salaries
CROSS JOIN seniorities
"""
duckdb.sql(cross_join_query)

┌────────┬─────────────┬─────────────┬───────────┐
│ salary │ employee_id │ employee_id │ seniority │
│ int64  │    int64    │    int64    │  varchar  │
├────────┼─────────────┼─────────────┼───────────┤
│   2000 │           1 │           1 │ 2ans      │
│   2500 │           2 │           1 │ 2ans      │
│   2200 │           3 │           1 │ 2ans      │
│   2000 │           1 │           2 │ 4ans      │
│   2500 │           2 │           2 │ 4ans      │
│   2200 │           3 │           2 │ 4ans      │
└────────┴─────────────┴─────────────┴───────────┘

## Cross join avec un filtre = inner join !

Si on décide maintenant d'appliquer un filtre avec la clause WHERE, on peut se limiter aux entrées pour lesquelles les deux colonnes employee_id sont égales:

In [5]:
cross_join_query_where = """
SELECT * FROM salaries
CROSS JOIN seniorities
WHERE salaries.employee_id = seniorities.employee_id
"""
duckdb.sql(cross_join_query_where)

┌────────┬─────────────┬─────────────┬───────────┐
│ salary │ employee_id │ employee_id │ seniority │
│ int64  │    int64    │    int64    │  varchar  │
├────────┼─────────────┼─────────────┼───────────┤
│   2000 │           1 │           1 │ 2ans      │
│   2500 │           2 │           2 │ 4ans      │
└────────┴─────────────┴─────────────┴───────────┘

<img src="images/cross_join_after_filter.png" />

Un exemple plus propre qui ne garde qu'une seule colonne "employee_id":

## Plus clean:

In [6]:
cross_join_query_where = """
SELECT salary, salaries.employee_id, seniority 
FROM salaries
CROSS JOIN seniorities
WHERE salaries.employee_id = seniorities.employee_id
"""
duckdb.sql(cross_join_query_where)

┌────────┬─────────────┬───────────┐
│ salary │ employee_id │ seniority │
│ int64  │    int64    │  varchar  │
├────────┼─────────────┼───────────┤
│   2000 │           1 │ 2ans      │
│   2500 │           2 │ 4ans      │
└────────┴─────────────┴───────────┘

L'utilisation de la syntaxe de l'INNER JOIN nous donne exactement le même résultat que le CROSS JOIN avec un FILTRE:

In [7]:
cross_join_query_where = """
SELECT salary, salaries.employee_id, seniority 
FROM salaries
INNER JOIN seniorities
ON salaries.employee_id = seniorities.employee_id
"""
duckdb.sql(cross_join_query_where)

┌────────┬─────────────┬───────────┐
│ salary │ employee_id │ seniority │
│ int64  │    int64    │  varchar  │
├────────┼─────────────┼───────────┤
│   2000 │           1 │ 2ans      │
│   2500 │           2 │ 4ans      │
└────────┴─────────────┴───────────┘

⇒ En définitive, toutes les sortes de joins peuvent être ramenées à un produit cartésien auquel on applique un filtre.


Dans le prochain notebook, on va voir quelques exemples d'INNER JOIN